In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use('ggplot')    # стиль графиков
%matplotlib inline
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from fake_useragent import UserAgent
from collections import defaultdict
import json
parsing_dict = {'Россия': 'russia', 'Великобритания': 'united-kingdom', 'США': 'united-states', 'Германия': 'germany',
                    'Франция': 'france', 'Япония': 'japan'}
for country in parsing_dict:
    if country == "Япония":
        url = f'https://ru.investing.com/equities/{parsing_dict[country]}'
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
tree = response.content
df = pd.read_html(tree, encoding='utf-8', thousands = '.')
df = df[0]
df = df.drop(['Unnamed: 0'], axis=1)
df = df.drop(['Unnamed: 9'], axis=1)
columns_names = ['Название компании', 'Текущая цена', 'Максимальная цена за сегодня',
                     'Минимальная цена за сегодня',
                     'Изменение в базисных пунктах', 'Изменение в процентах', 'Объем торгов', 'Время']
df.columns = columns_names
dates = []
df['Текущая цена']= df['Текущая цена'].str.replace('.', '')
df['Текущая цена']= df['Текущая цена'].str.replace(',', '.')
df['Максимальная цена за сегодня']= df['Максимальная цена за сегодня'].str.replace('.', '')
df['Максимальная цена за сегодня']= df['Максимальная цена за сегодня'].str.replace(',', '.')
df['Минимальная цена за сегодня']= df['Минимальная цена за сегодня'].str.replace('.', '')
df['Минимальная цена за сегодня']= df['Минимальная цена за сегодня'].str.replace(',', '.')
lst = []
for i in df['Объем торгов']:
    i = i.replace(',', '.')
    if i[-1] == 'K':
        i = float(i[:-1:]) * 1000
        lst.append(i)
    elif i[-1] == 'M':
        i = float(i[:-1:]) * 1000000
        lst.append(i)
    elif i[-1] == 'B':
        lst.append(None)

df["Объём торгов (для графиков)"] = lst
df['Объём торгов (для графиков)'] = df['Объём торгов (для графиков)']/ 1000
df.dropna(axis = 0, inplace=True)
df['Текущая цена']= df['Текущая цена'].astype(float)
df.to_csv('Shares.csv', 'w', index = False, index_label = False)
#Ссылки на каждую акцию
links_tree = BeautifulSoup(tree)
links_tree = links_tree.find_all('td', {'class': 'bold left noWrap elp plusIconTd'})
d = defaultdict()
for link in range(1,len(links_tree)+1):
    d[link] = 'https://ru.investing.com' + links_tree[link-1].a.get('href')
with open('links.json', 'w') as json_file:
    json.dump(d, json_file)
        
#Функция для вывода динамики по акции
def dynamics_func(current_share):
    with open('links.json') as json_file:
        dictionary = json.load(json_file)
    #Парсинг таблицы
        url = dictionary[f'{current_share}']
        response = requests.get(url)
        tree = BeautifulSoup(response.content, features="lxml")
        data = tree.find('div', {'class': 'instrument-page_section__79xMl'})
        data = tree.find('ul', {'class': 'list_list--horizontal__3fUDd navbar_navbar-sub__1_y2w'})
        data_list = []
        for j in data:
            data_list.append(j)
        data = 'https://ru.investing.com' + data_list[2].a.get('href')
        response = requests.get(data, headers={'User-Agent': UserAgent().chrome})
        tree = response.content
        df = pd.read_html(tree, attrs={'id': 'curr_table'}, decimal=',', thousands='.')
        df = df[0]
        # Преобразование даты
        dates = []
        for ind in range(len(df['Дата'])):
            if len(df['Дата'][ind].astype(str)[:-4]) == 4:
                date = df['Дата'][ind].astype(str)[:-4][:2]
                month = df['Дата'][ind].astype(str)[:-4][-2:]
                final_date = date + '.' + month
                dates.append(final_date)
            elif len(df['Дата'][ind].astype(str)[:-4]) == 3:
                date = '0' + df['Дата'][ind].astype(str)[:-4][:1]
                month = df['Дата'][ind].astype(str)[:-4][-2:]
                final_date = date + '.' + month
                dates.append(final_date)
        df['Дата'] = dates
        return df
    

In [2]:
dynamics_func(1)

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,16.06,31.88,32.43,32.79,31.74,"38,50M","-1,69%"
1,15.06,32.43,32.79,33.29,32.20,"53,48M","-0,73%"
2,14.06,32.67,32.34,32.83,32.34,"11,05M","1,12%"
3,11.06,32.30,32.23,32.58,32.12,"19,24M","0,07%"
4,10.06,32.28,32.40,32.61,32.14,"16,69M","-0,38%"
5,09.06,32.40,32.70,32.88,32.10,"36,52M","-1,21%"
6,08.06,32.80,32.55,32.90,32.45,"26,11M","0,99%"
7,07.06,32.48,31.91,32.63,31.90,"26,44M","1,79%"
8,04.06,31.91,32.30,32.40,31.55,"26,28M","-1,05%"
9,03.06,32.25,31.99,32.55,31.70,"62,14M","1,17%"
